In [11]:
import subprocess
import sys

def install_package(package):
    """Install package using pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} installed successfully!")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package}")

# Check and install dependencies
try:
    import qrcode
    from PIL import Image
    print("Dependencies are already available!")
except ImportError:
    print("Installing dependencies...")
    install_package("qrcode[pil]")
    install_package("pillow")

    # Re-import after installation
    try:
        import qrcode
        from PIL import Image
        print("Dependencies installed and imported successfully!")
    except ImportError as e:
        print(f"Error: {e}")
        print("Try running in terminal: pip install qrcode[pil] pillow")

import os
import argparse

def generate_qr_code(url, filename="qr_code.png", size=10, border=4, error_correction='L', fill_color="black", back_color="white", verbose=True):
    """
    Generate QR code from the given URL/link

    Parameters:
    url (str): The URL to convert to QR code
    filename (str): Output file name (default: qr_code.png)
    size (int): Box size of QR code (default: 10)
    border (int): Border size of QR code (default: 4)
    error_correction (str): Error correction level ('L', 'M', 'Q', 'H') (default: 'L')
    fill_color (str): Fill color (default: black)
    back_color (str): Background color (default: white)
    verbose (bool): Print status messages (default: True)

    Returns:
    bool: True if successful, False if failed
    """
    try:
        error_levels = {
            'L': qrcode.constants.ERROR_CORRECT_L,
            'M': qrcode.constants.ERROR_CORRECT_M,
            'Q': qrcode.constants.ERROR_CORRECT_Q,
            'H': qrcode.constants.ERROR_CORRECT_H
        }
        if error_correction not in error_levels:
            raise ValueError("Invalid error correction level. Choose from 'L', 'M', 'Q', 'H'.")

        # Create QR code instance
        qr = qrcode.QRCode(
            version=None,  # Auto-detect version
            error_correction=error_levels[error_correction],
            box_size=size,
            border=border,
        )

        # Add data (URL) to QR code
        qr.add_data(url)
        qr.make(fit=True)

        # Create QR code image
        img = qr.make_image(fill_color=fill_color, back_color=back_color)

        # Save image
        img.save(filename)

        if verbose:
            print(f"QR Code generated successfully!")
            print(f"File saved as: {filename}")
            print(f"URL: {url}")

        return True

    except Exception as e:
        if verbose:
            print(f"Error: {str(e)}")
        return False

def generate_qr_with_logo(url, logo_path, filename="qr_code_with_logo.png", size=10, border=4, error_correction='H', fill_color="black", back_color="white", logo_scale=0.2, verbose=True):
    """
    Generate QR code with a logo in the center

    Parameters:
    url (str): The URL to convert to QR code
    logo_path (str): Path to the logo file
    filename (str): Output file name
    size (int): Box size of QR code
    border (int): Border size of QR code
    error_correction (str): Error correction level ('L', 'M', 'Q', 'H') (default: 'H')
    fill_color (str): Fill color (default: black)
    back_color (str): Background color (default: white)
    logo_scale (float): Scale of logo relative to QR size (default: 0.2)
    verbose (bool): Print status messages (default: True)

    Returns:
    bool: True if successful, False if failed
    """
    try:
        if not os.path.exists(logo_path):
            raise FileNotFoundError(f"Logo file not found: {logo_path}")

        error_levels = {
            'L': qrcode.constants.ERROR_CORRECT_L,
            'M': qrcode.constants.ERROR_CORRECT_M,
            'Q': qrcode.constants.ERROR_CORRECT_Q,
            'H': qrcode.constants.ERROR_CORRECT_H
        }
        if error_correction not in error_levels:
            raise ValueError("Invalid error correction level. Choose from 'L', 'M', 'Q', 'H'.")

        # Create base QR code
        qr = qrcode.QRCode(
            version=None,
            error_correction=error_levels[error_correction],
            box_size=size,
            border=border,
        )

        qr.add_data(url)
        qr.make(fit=True)

        # Create QR code image
        qr_img = qr.make_image(fill_color=fill_color, back_color=back_color).convert("RGBA")

        # Open and resize logo
        logo = Image.open(logo_path).convert("RGBA")

        # Calculate logo size
        qr_width, qr_height = qr_img.size
        logo_size = int(min(qr_width, qr_height) * logo_scale)

        # Resize logo
        logo = logo.resize((logo_size, logo_size), Image.Resampling.LANCZOS)

        # Calculate center position for logo
        logo_pos = ((qr_width - logo_size) // 2, (qr_height - logo_size) // 2)

        # Paste logo onto QR code
        qr_img.paste(logo, logo_pos, logo)

        # Save image
        qr_img.save(filename)

        if verbose:
            print(f"QR Code with logo generated successfully!")
            print(f"File saved as: {filename}")
            print(f"URL: {url}")

        return True

    except Exception as e:
        if verbose:
            print(f"Error: {str(e)}")
        return False

def batch_generate_qr(urls_list, prefix="qr_", size=10, border=4, error_correction='L', fill_color="black", back_color="white", verbose=True):
    """
    Generate multiple QR codes from a list of URLs

    Parameters:
    urls_list (list): List of URLs
    prefix (str): Prefix for file names
    size (int): Box size of QR code
    border (int): Border size of QR code
    error_correction (str): Error correction level
    fill_color (str): Fill color
    back_color (str): Background color
    verbose (bool): Print status messages

    Returns:
    int: Number of QR codes successfully created
    """
    success_count = 0

    for i, url in enumerate(urls_list, 1):
        filename = f"{prefix}{i}.png"
        if generate_qr_code(url, filename, size, border, error_correction, fill_color, back_color, verbose):
            success_count += 1
        if verbose:
            print("-" * 50)

    if verbose:
        print(f"Total QR codes generated: {success_count}/{len(urls_list)}")
    return success_count

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Advanced QR Code Generator")
    parser.add_argument("--url", type=str, help="URL to generate QR for")
    parser.add_argument("--urls-file", type=str, help="File containing list of URLs (one per line) for batch generation")
    parser.add_argument("--logo", type=str, help="Path to logo file for embedding")
    parser.add_argument("--output", type=str, default="qr_code.png", help="Output file name")
    parser.add_argument("--size", type=int, default=10, help="Box size (default: 10)")
    parser.add_argument("--border", type=int, default=4, help="Border size (default: 4)")
    parser.add_argument("--error-correction", type=str, default='L', choices=['L', 'M', 'Q', 'H'], help="Error correction level (default: L)")
    parser.add_argument("--fill-color", type=str, default="black", help="Fill color (default: black)")
    parser.add_argument("--back-color", type=str, default="white", help="Background color (default: white)")
    parser.add_argument("--logo-scale", type=float, default=0.2, help="Logo scale relative to QR size (default: 0.2)")
    parser.add_argument("--prefix", type=str, default="qr_", help="Prefix for batch file names (default: qr_)")
    parser.add_argument("--quiet", action="store_true", help="Suppress status messages")

    args, unknown = parser.parse_known_args()
    verbose = not args.quiet

    if args.url is None and args.urls_file is None:
        print("No URL or URLs file provided. Entering interactive mode.")

        mode = input("Choose mode (1: single URL, 2: batch from file): ").strip()

        if mode == "1":
            args.url = input("Enter URL: ").strip()
            if not args.url:
                print("URL is required.")
                sys.exit(1)

            args.logo = input("Enter logo path (optional): ").strip() or None
            args.output = input(f"Enter output filename (default: {args.output}): ").strip() or args.output
            size_str = input(f"Enter box size (default: {args.size}): ").strip()
            args.size = int(size_str) if size_str else args.size
            border_str = input(f"Enter border size (default: {args.border}): ").strip()
            args.border = int(border_str) if border_str else args.border
            args.error_correction = input(f"Enter error correction level (L/M/Q/H, default: {args.error_correction}): ").strip() or args.error_correction
            args.fill_color = input(f"Enter fill color (default: {args.fill_color}): ").strip() or args.fill_color
            args.back_color = input(f"Enter background color (default: {args.back_color}): ").strip() or args.back_color
            if args.logo:
                logo_scale_str = input(f"Enter logo scale (default: {args.logo_scale}): ").strip()
                args.logo_scale = float(logo_scale_str) if logo_scale_str else args.logo_scale

        elif mode == "2":
            args.urls_file = input("Enter path to URLs file: ").strip()
            if not args.urls_file:
                print("URLs file path is required.")
                sys.exit(1)

            args.prefix = input(f"Enter prefix for batch files (default: {args.prefix}): ").strip() or args.prefix
            size_str = input(f"Enter box size (default: {args.size}): ").strip()
            args.size = int(size_str) if size_str else args.size
            border_str = input(f"Enter border size (default: {args.border}): ").strip()
            args.border = int(border_str) if border_str else args.border
            args.error_correction = input(f"Enter error correction level (L/M/Q/H, default: {args.error_correction}): ").strip() or args.error_correction
            args.fill_color = input(f"Enter fill color (default: {args.fill_color}): ").strip() or args.fill_color
            args.back_color = input(f"Enter background color (default: {args.back_color}): ").strip() or args.back_color

        else:
            print("Invalid mode selected.")
            sys.exit(1)

    if args.urls_file:
        try:
            with open(args.urls_file, 'r') as f:
                urls = [line.strip() for line in f if line.strip()]
            if not urls:
                print("No URLs found in file.")
                sys.exit(1)
            batch_generate_qr(urls, args.prefix, args.size, args.border, args.error_correction, args.fill_color, args.back_color, verbose)
        except Exception as e:
            print(f"Error reading file: {e}")
    elif args.url:
        if args.logo:
            generate_qr_with_logo(args.url, args.logo, args.output, args.size, args.border, args.error_correction, args.fill_color, args.back_color, args.logo_scale, verbose)
        else:
            generate_qr_code(args.url, args.output, args.size, args.border, args.error_correction, args.fill_color, args.back_color, verbose)
    else:
        print("No valid input provided.")
        parser.print_help()
        sys.exit(1)

Dependencies are already available!
No URL or URLs file provided. Entering interactive mode.
Choose mode (1: single URL, 2: batch from file): 1
Enter URL: https://www.youtube.com/watch?v=fPO76Jlnz6c&list=RD-_Av7re3Ce8&index=26
Enter logo path (optional): 
Enter output filename (default: qr_code.png): 
Enter box size (default: 10): 10
Enter border size (default: 4): 4
Enter error correction level (L/M/Q/H, default: L): M
Enter fill color (default: black): red
Enter background color (default: white): blue
QR Code generated successfully!
File saved as: qr_code.png
URL: https://www.youtube.com/watch?v=fPO76Jlnz6c&list=RD-_Av7re3Ce8&index=26


Note: ignored unknown args: ['-f', '/root/.local/share/jupyter/runtime/kernel-8ff0fc53-04a6-404b-b39b-35f0af8b8c1a.json']
QR code generated successfully.
Saved to: qr_code.png
URL encoded: https://www.youtube.com/watch?v=pvrkfb1C4tE&list=RD-_Av7re3Ce8&index=14


In [ ]:
from google.colab import drive
drive.mount('/content/drive')